# Chapter 13: Fearless Concurrency
\n**Note:** This notebook will install Rust in your Colab environment. Run the setup cell first!\n
Learn how Rust enables safe concurrent programming.

In [ ]:
%%bash
# Install Rust in Colab (run this cell first!)
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
source $HOME/.cargo/env
rustc --version

## Creating Threads

In [ ]:
use std::thread;
use std::time::Duration;

fn main() {
    thread::spawn(|| {
        for i in 1..10 {
            println!("hi number {} from the spawned thread!", i);
            thread::sleep(Duration::from_millis(1));
        }
    });
    
    for i in 1..5 {
        println!("hi number {} from the main thread!", i);
        thread::sleep(Duration::from_millis(1));
    }
}

## Waiting for Threads with join

In [ ]:
use std::thread;

fn main() {
    let handle = thread::spawn(|| {
        for i in 1..10 {
            println!("number {}", i);
        }
    });
    
    handle.join().unwrap();  // Wait for thread to finish
    println!("Thread finished");
}

## Message Passing with Channels

In [ ]:
use std::sync::mpsc;
use std::thread;

fn main() {
    let (tx, rx) = mpsc::channel();
    
    thread::spawn(move || {
        let val = String::from("hi");
        tx.send(val).unwrap();
    });
    
    let received = rx.recv().unwrap();
    println!("Got: {}", received);
}

## Shared State with Mutex

In [ ]:
use std::sync::{Arc, Mutex};
use std::thread;

fn main() {
    let counter = Arc::new(Mutex::new(0));
    let mut handles = vec![];
    
    for _ in 0..10 {
        let counter = Arc::clone(&counter);
        let handle = thread::spawn(move || {
            let mut num = counter.lock().unwrap();
            *num += 1;
        });
        handles.push(handle);
    }
    
    for handle in handles {
        handle.join().unwrap();
    }
    
    println!("Result: {}", *counter.lock().unwrap());
}

## Key Takeaways

- Use `thread::spawn` to create threads
- Channels enable safe message passing
- `Mutex<T>` and `Arc<T>` enable shared state concurrency
- Rust's type system prevents data races at compile time